In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')
train_sequences = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')

In [3]:
train_labels.head()

,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793


In [4]:
train_sequences.head()

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...,>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*A...
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA..."


In [5]:
import pandas as pd

# Extract IDs with NaN values in train_labels
nan_ids = train_labels.loc[train_labels[['x_1', 'y_1', 'z_1']].isna().any(axis=1), 'ID']

# Remove the last underscore and number (extract base ID)
nan_base_ids = nan_ids.str.rsplit('_', n=1).str[0].unique()

# Drop rows from train_labels where base ID matches
train_labels = train_labels[~train_labels["ID"].str.rsplit('_', n=1).str[0].isin(nan_base_ids)]

# Drop rows from train_sequences where base ID matches
train_sequences = train_sequences[~train_sequences["target_id"].isin(nan_base_ids)]

# Reset index if needed
train_labels.reset_index(drop=True, inplace=True)
train_sequences.reset_index(drop=True, inplace=True)

# **📌 Step 1: Improving the GNN Model**
### **1.1 Use a Graph Transformer Instead of GCN**
Instead of a basic **Graph Convolutional Network (GCN)**, we can use a **Graph Transformer**, which is better at capturing **long-range interactions** in RNA.

### **🔹 Code: Graph Transformer Model**


In [6]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.0 MB/s eta 0:00:00


In [7]:
import torch
import torch.nn as nn
from torch_geometric.nn import TransformerConv

class RNA_Transformer_GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=3):
        super(RNA_Transformer_GNN, self).__init__()
        
        self.layers = nn.ModuleList()
        for _ in range(num_layers):
            self.layers.append(TransformerConv(hidden_dim, hidden_dim, heads=4))

        self.fc_input = nn.Linear(input_dim, hidden_dim)
        self.fc_output = nn.Linear(hidden_dim, 3)  # Predict (x, y, z)

    def forward(self, x, edge_index):
        x = self.fc_input(x)
        for layer in self.layers:
            x = torch.relu(layer(x, edge_index))
        return self.fc_output(x)

# Model initialization
model = RNA_Transformer_GNN(input_dim=4, hidden_dim=128)
print(model)

RNA_Transformer_GNN(
  (layers): ModuleList(
    (0-2): 3 x TransformerConv(128, 128, heads=4)
  )
  (fc_input): Linear(in_features=4, out_features=128, bias=True)
  (fc_output): Linear(in_features=128, out_features=3, bias=True)
)


🔹 **Why is this better?**
- **TransformerConv** captures **long-range dependencies** in RNA (better than GCN).  
- **Multi-head attention** improves structural context awareness.  

---

### **1.2 Use MSA Embeddings Instead of One-Hot Encoding**
We can replace **one-hot encoding** with **MSA Transformer embeddings** (e.g., from ESM-1b or RNA-specific models).

### **🔹 Code: Extract MSA Embeddings**

In [8]:
!pip install multimolecule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 11.4 MB/s eta 0:00:00


In [9]:
#divide long input string into chunks

def chunk_sequence(sequence, max_length=1024, overlap=128):
    """
    Splits a sequence into overlapping chunks.
    
    Args:
        sequence (str): RNA sequence (ACGU).
        max_length (int): Maximum model token length.
        overlap (int): Number of overlapping tokens.

    Returns:
        list of str: List of sequence chunks.
    """
    chunks = []
    start = 0
    while start < len(sequence):
        chunk = sequence[start:start + max_length]
        chunks.append(chunk)
        start += max_length - overlap  # Move forward with overlap
        if len(chunk) < max_length:
            break  # Stop if last chunk is short
    return chunks

In [10]:
# from transformers import EsmTokenizer, EsmForMaskedLM

# # Load pretrained ESM model
# tokenizer = EsmTokenizer.from_pretrained("facebook/esm1b_t33_650M_UR50S")
# model = EsmForMaskedLM.from_pretrained("facebook/esm1b_t33_650M_UR50S", output_hidden_states=True)

# print("Max Position Embeddings:", model.config.max_position_embeddings)

from multimolecule import RnaTokenizer, RnaFmModel
tokenizer = RnaTokenizer.from_pretrained("multimolecule/rnafm")
model = RnaFmModel.from_pretrained("multimolecule/rnafm")

def get_msa_embedding(sequence):
    """Generate MSA embedding for RNA sequence"""
    # Chunk it
    chunks = chunk_sequence(sequence)
    print(chunks)

    # Tokenize each chunk
    tokenized_chunks = [tokenizer(chunk, return_tensors="pt", add_special_tokens=False) for chunk in chunks]

    #inputs = tokenizer(sequence, return_tensors="pt", add_special_tokens=False)
    # print("Token IDs:", inputs["input_ids"])
    # print("Vocab size:", model.config.vocab_size)

    # if "position_ids" in inputs and (inputs["position_ids"] >= model.config.max_position_embeddings).any():
    #     raise ValueError("Position ID out of range!")
    
    with torch.no_grad():
        # Get model output (logits and hidden states)
        # print(f"Processing sequence: {sequence}")
        # print("Tokenized input_ids:", inputs["input_ids"])
        # print("Input shape:", inputs["input_ids"].shape)
        
        #output = model(**inputs)
        embeddings = [model(**chunk).last_hidden_state.squeeze(0) for chunk in tokenized_chunks]
        
        # Accessing the last hidden state from the output
    return embeddings

# Example usage
msa_embedding = get_msa_embedding("GGGAAACCC")
print(msa_embedding[0].shape)  # Expected: (seq_length, embedding_dim)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/398M [00:00<?, ?B/s]

Some weights of RnaFmModel were not initialized from the model checkpoint at multimolecule/rnafm and are newly initialized: ['rnafm.pooler.dense.bias', 'rnafm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['GGGAAACCC']
torch.Size([9, 640])


🔹 **Why is this better?**
- **MSA embeddings** capture **evolutionary conservation** from large protein/RNA databases.
- **Pretrained models** (like ESM or RNA-specific transformers) generalize well to new sequences.

🔹 4. Graph-Aware Aggregation (For Graph-Based Models)
Instead of treating the sequence as linear, map it to a graph first and aggregate node features:

In [11]:
def create_chunk_graph(chunk_embeddings, window_size=3):
    """
    This function takes the chunk embeddings and generates a graph structure.
    Each chunk will be a node, and edges will connect consecutive chunks.
    """
    num_chunks = len(chunk_embeddings)

    # Create edge_index to connect consecutive chunks (nodes)
    edges = []
    for i in range(num_chunks - 1):
        edges.append([i, i + 1])  # Edge between consecutive nodes

    # Convert to tensor form
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # Shape: [2, num_edges]

    # Create the graph structure
    graph = Data(x=chunk_embeddings, edge_index=edge_index)
    return graph

In [13]:
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data

class GNN_Aggregator(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.conv1 = pyg_nn.GraphConv(hidden_dim, hidden_dim)
        self.conv2 = pyg_nn.GraphConv(hidden_dim, hidden_dim)

    def forward(self, graph):
        x, edge_index = graph.x, graph.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x  # (num_nodes, hidden_dim)

# Example usage
rna_graph = create_chunk_graph(msa_embedding)  # Assign embeddings as node features
# print(f"rna_graph.x shape: {rna_graph.x.shape}")
# print(f"rna_graph.edge_index shape: {rna_graph.edge_index.shape}")
print(f"rna_graph.edge_index: {rna_graph.edge_index}")

gnn_aggregator = GNN_Aggregator(hidden_dim=640)
final_representation = gnn_aggregator(rna_graph)
print(final_representation.shape)

rna_graph.edge_index: tensor([], dtype=torch.int64)


ValueError: Expected 'edge_index' to be two-dimensional (got 1 dimensions)

In [ ]:
from torch_geometric.data import Data

def create_rna_graph(msa_embedding):
    # Assume a simple chain connectivity for now
    num_nodes = msa_embedding.size(0)
    # Connect each nucleotide to its next one: edge from i to i+1
    edge_index = torch.tensor([[i, i+1] for i in range(num_nodes-1)] +
                              [[i+1, i] for i in range(num_nodes-1)], dtype=torch.long).t().contiguous()
    return Data(x=msa_embedding, edge_index=edge_index)

# Example usage:
rna_graph = create_rna_graph(final_representation)
print("Graph node features shape:", rna_graph.x.shape)
print("Graph edge_index shape:", rna_graph.edge_index.shape)

### **1.3 Get Graphs from MSA Embeddings of Training Data**

In [ ]:
from tqdm import tqdm

graph_list=[]

for idx, row in tqdm(train_sequences.iterrows()):
    target_id = row['target_id']
    sequence = row['sequence']

    # Filter coordinates for this target
    target_coords = train_labels[train_labels['ID'].str.startswith(target_id)]
    
    if len(target_coords) == 0:
        print(f"No coordinates found for {target_id}, skipping")
        continue

    seq_msa = get_msa_embedding(sequence)
    graph = create_rna_graph(seq_msa)

    graph_list.append(graph)

print(graph_data_list[0])
print(len(graph_data_list))

# **📌 Step 2: Physics-Based Refinement**
Even if ML predicts **good** RNA structures, **fine-tuning with physics-based methods** improves accuracy.

---

## **2.1 Use Rosetta for Energy Minimization**
Rosetta is a **powerful molecular modeling software** that refines predicted structures.

### **🔹 Steps for Using Rosetta:**
1. **Convert ML predictions to PDB format**  
2. **Run Rosetta minimization** to relax the structure  
3. **Extract refined coordinates** for submission  

### **🔹 Code: Convert Predictions to PDB Format**


In [48]:
def write_pdb(predictions, sequence, output_file="rna_model.pdb"):
    """Writes RNA coordinates to PDB format"""
    with open(output_file, "w") as f:
        for i, (x, y, z) in enumerate(predictions):
            resname = sequence[i]
            f.write(f"HETATM {i+1:4d}  C1'  {resname} A   {i+1:3d}     {x:8.3f}{y:8.3f}{z:8.3f}  1.00  0.00\n")
    print(f"PDB file saved: {output_file}")

# Example usage
dummy_predictions = np.random.rand(10, 3)  # Example (x, y, z) predictions
write_pdb(dummy_predictions, "GGGAAACCC")

IndexError: string index out of range

### **🔹 Command: Run Rosetta Refinement**
After converting ML predictions to PDB format, we refine them using Rosetta:

In [ ]:
rosetta_scripts.default.linuxgccrelease -s rna_model.pdb -parser:protocol rna_minimize.xml

### **2.2 Use Molecular Dynamics (MD) Simulation**
Molecular Dynamics (MD) uses physics-based **force fields** to refine RNA structures.

### **🔹 Steps for MD Refinement**
1. **Use OpenMM / GROMACS** to set up RNA simulation.  
2. **Minimize energy** and relax RNA structure.  
3. **Extract final coordinates** after stabilization.  

### **🔹 Code: Run MD Simulation (OpenMM)**

In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

def run_md_simulation(pdb_file, output_file="rna_relaxed.pdb"):
    """Performs energy minimization using OpenMM"""
    pdb = PDBFile(pdb_file)
    forcefield = ForceField('amber99sb.xml', 'tip3p.xml')

    system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, constraints=HBonds)
    integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(pdb.positions)
    
    # Minimize energy
    simulation.minimizeEnergy()
    
    # Save refined structure
    positions = simulation.context.getState(getPositions=True).getPositions()
    PDBFile.writeFile(simulation.topology, positions, open(output_file, 'w'))
    print(f"Refined PDB saved: {output_file}")

# Example usage
run_md_simulation("rna_model.pdb")

🔹 **Why is this useful?**
- Ensures the RNA model follows **physical constraints**.
- Reduces **structural artifacts** from ML predictions.

---

# **📌 Step 3: Combining Everything for Best Accuracy**
We now **combine ML + MSA + Physics-based methods** to generate final predictions.

### **🔹 Final Workflow**
1. **Train ML Model (GNN/Transformer)**
   - Inputs: RNA sequence + MSA features.
   - Outputs: Predicted (x, y, z) coordinates.
2. **Refine with Rosetta**
   - Convert ML output to **PDB format**.
   - Run **energy minimization**.
3. **Further Relaxation with Molecular Dynamics (MD)**
   - Ensure **stable structure** with OpenMM/GROMACS.
4. **Format Predictions for Submission**
   - Extract final (x, y, z) from PDB.

---

# **🚀 Next Steps**
✅ **Train ML model on full dataset (GNN + Transformer)**  
✅ **Benchmark ML vs. Rosetta vs. MD Refinement**  
✅ **Optimize final submission** with hybrid ML + physics approach  